---
title: "LC121: Best Time to Buy and Sell Stock"
author: "Vahram Poghosyan"
date: "2024-05-23"
categories: ["Leetcode", "Algorithms", "Dyanmic Progamming"]
format:
  html:
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

# Problem Statement

We are given an array of `prices` where `prices[i]` is the price of a given stock on the `i`-th day.

We want to maximize our profit by choosing a single day to buy one stock and choosing a different day in the future to sell that stock. Return the maximum profit you can achieve from this transaction. If you cannot achieve any profit, return `0`.

**Example 1**

```
Input: prices = [7,1,5,3,6,4]
Output: 5
```

**Explanation** 

Buy on day 2 (price = 1) and sell on day 5 (price = 6), profit = 6-1 = 5.
Note that buying on day 2 and selling on day 1 is not allowed because you must buy before you sell

**Example 2:**

```
Input: prices = [7,6,4,3,1]
Output: 0
```

**Explanation** 

In this case, no transactions are done and the max profit = 0.

# Brute-Force Solution

Consider each viable pair of days. It's easy to see that this leads to time complexity $O(n^2)$ because, for each possible day `i` that we choose to buy the stock on, there are $n-i$ possible days that we can sell it on. Since there are $n$ choices for which day to buy, the number of total pairs has a leading term of $n(n-1)$, so it's quadratic in $n$. 

We can also think of choosing a subset of size $2$ and discarding those which have a reverse order of days. This essentially means choosing a subset of size $2$ without order (since each pair is either in the correct order or not, and we only count the one that is), so ${O \left ({n \choose 2} \right )}$ which is, of course, $O(n^2)$.

# Optimal Solution - DP 

We can solve this problem in a single pass, achieving $O(n)$ complexity by using DP with tabulation similar to [Kadane's algorithm](https://en.wikipedia.org/wiki/Maximum_subarray_problem#Kadane's_algorithm) which is explored, in detail, in this [post](./lc53_maximum_subarray.ipynb). In this one, we offer the solution and part of its **proof of correctness**. In the [Maximum Subarray post](./lc53_maximum_subarray.ipynb) we only go over the intuition behind the solution and do not offer proof. These two posts are meant to complement one another.

In [ ]:
#| code-false: false

prices = [7,1,5,3,6,4]

In [ ]:
#| code-fold: false
min_price = float("inf")
max_profit = 0 

for i in range(len(prices)):
    if prices[i] < min_price:
        min_price = prices[i]
    elif prices[i] - min_price > max_profit:
        max_profit = prices[i] - min_price

print(max_profit)

5


# Proof of Correctness

It's easy to see that, at the end of each iteration `k`, the variable `min_price` holds the lowest price dip in the stock up to, and including, the index `k`. The variable `min_price` is what's known as a **loop invariant**. Showing something is a loop invariant is a lot like proving the **inductive step** of a **proof by induction**.

It can be shown that `max_profit` is yet another loop invariant, and that it holds the maximum profit up to, and including, the index `k`. This is the solution to the `k+1`-sized sub-problem (as index `0` corresponds to a size `1` sub-problem).  

## Proof of Invariance of `max_profit`

The loop above does one of only two things at some iteration `k`: either it updates `min_price` or it doesn't. These are, obviously, exclusive scenarios. 

Suppose it *doesn't* update `min_price` and let's label this case as **Case 1**. In this case `prices[k] > min_price`. The first loop invariant, `min_price` holds the lowest price dip up to, and including, the index `k` (we'll leave the proof of this to the reader). In this first case, the difference of `prices[k]` and `min_price` is calculated and `max_profit` is updated *if and only if* the difference is greater than the value of `max_profit` at the end of the previous iteration. This guarantees that `max_profit` still holds the maximum profit up to, and including, index `k` at the end of the current iteration. 

In the other opposite case (**Case 2**), when the loop *does* update `min_price` at the current iteration, it proceeds to the subsequent iteration with `max_profit` *still* holding the maximum profit up to, and including, only index `k` (not `k+1`). If `prices[k+1]` is, again, less than `min_price` (which is just `prices[k]` at that point) then the loop just goes on updating `min_price` until it encounters an uptick in the price. Note that if the prices just keep decreasing until the very end, then the proof is complete. Since the prices just keep decreasing monotonically, `max_profit` (which holds the answer up to index `k`) is actually the final answer -- The perfect time to sell, if the stock just keeps crashing after some index `k`, *would* be at index `k`. So, suppose we're in the interesting case when there is no monotonic crash till the very end. Then, at some future iteration corresponding to index `j` (where `j` > `k+2` because the `k+1`-th index which, by assumption, represents a price dip corresponds to iteration `k+2`), we have `prices[j] > min_price`. But notice that this puts us, again, in the familiar **Case 1** whereby `min_price` does get updated. We've already shown that `max_profit` holds the maximum profit up to, and including, the current index in that case.

Hence, `max_profit` is a loop invariant in both of the cases above (which constitute the set of *all* possible cases). Therefore, at the last iteration of the loop, `max_profit` holds the maximum profit up to, and including, the last index `n-1` (for an `n`-sized problem). In other words, it holds the final solution.

# Summary

In this notebook, we explored the problem of finding the best time to buy and sell stock to maximize profit. We started with a brute-force solution that has a time complexity of \(O(n^2)\). Then, we offered an optimal solution using dynamic programming with a time complexity of \(O(n)\) without discussing the process of coming up with the algorithm (the intuition is discussed more extensively in the post on [Maximum Subarray](./lc53_maximum_subarray.ipynb)). The focus of this post was the proof of correctness of the optimal solution.